# Assigment 4

In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt

from typing import Any, Callable

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            
            # Add the term conj(f)^n*ui - conj(g)^n*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)
    
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

In [5]:
R = align_field(v, f, tt, cf, c, 1e6)
plot_mesh_field(v, f, R, cf)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Part 1 - Tangent vector fields for scalar field design

In [6]:
def construct_vector_field(
    vertices: list[Any],
    faces: list[Any],
    trig_trig_adj: list[Any],
    hard_id: list[Any],
    hard_value: list[Any]
) -> list[Any]:
    # Compute edges for each face
    edges1 = vertices[faces[:, 1], :] - vertices[faces[:, 0], :]
    edges2 = vertices[faces[:, 2], :] - vertices[faces[:, 0], :]
    
    # Compute local basis for each face     
    T1 = edges1 / np.linalg.norm(edges1, axis=1)[:, None]
    T2 = np.cross(T1, np.cross(T1, edges2))
    T2 /= np.linalg.norm(T2, axis=1)[:, None]
    
    # Ready Arrays
    data, ii, jj = list(), list(), list() 
    
    # Ready sets for bijectives
    m ,n = {}, {}
    
    # Track idx for m and n
    idx = 0
    
    # Loop over all the faces
    for face in range(faces.shape[0]):
        # Loop over edges
        for edge in range(3):
            opposite_face = trig_trig_adj[face, edge]
            if (opposite_face == -1) or (face > opposite_face):
                continue
            
            # Compute complex shared edge
            shared_edge = vertices[faces[face, (edge + 1) % 3], :] - vertices[faces[face, edge], :]
            
            vector_f = np.array([np.dot(shared_edge, T1[face, :]), np.dot(shared_edge, T2[face, :])])
            vector_f /= np.linalg.norm(vector_f)
            edge_f = vector_f[0] + vector_f[1] * 1J
            edge_f_conjugate = edge_f.conjugate()
                
            vector_g = np.array([np.dot(shared_edge, T1[opposite_face, :]), np.dot(shared_edge, T2[opposite_face, :])])
            vector_g /= np.linalg.norm(vector_g)
            edge_g = vector_g[0] + vector_g[1] * 1J
            edge_g_conjugate = edge_g.conjugate()
            
            # Add to prepared arrays
            data.append(1); ii.append(face); jj.append(face)
            data.append(1); ii.append(opposite_face); jj.append(opposite_face)
            
            data.append(-edge_f * edge_g_conjugate); ii.append(face); jj.append(opposite_face)
            data.append(edge_g * -edge_f_conjugate); ii.append(opposite_face); jj.append(face)
        
        # Skip if in hard constraint
        if face in hard_id:
            continue
        
        # Add to bijectives
        m[idx] = face
        n[face] = idx
        
        idx += 1
    
    # Construct Q matrix
    Q = sp.coo_matrix( (data, (ii, jj)), shape=(faces.shape[0], faces.shape[0])).asformat('csr')
    data, ii, jj = list(), list(), list() 
    
    
    # Energy Formulation [f(u) = A*Au - A*b = 0]
    u = np.zeros(faces.shape[0], dtype=complex)
    b = np.zeros((idx, 1), dtype=complex)
    
    # System assembly for hard constraints
    for i in range(idx):
        for j in Q.getrow(m[i]).nonzero()[1]:
            if j not in hard_id:
                data.append(Q[m[i], j])
                ii.append(i)
                jj.append(n[j])
            
        for ci in range(hard_id.shape[0]):
            face, vertex = hard_id[ci], hard_value[ci, :]
            
            # Project on to local frame
            constraint = np.dot(vertex, T1[face, :]) + np.dot(vertex, T2[face, :]) *1J
            
            # Update u and b
            u[face] = constraint
            b[i] -= constraint * Q[m[i], face] 
    
    # Solve linear system under hard constraints. 
    A = sp.coo_matrix( (data, (ii, jj)), shape=(idx, idx)).asformat('csr')
    temp_u = sp.linalg.spsolve(A, b)
    
    # Replace in U to maintain shape
    for i in range(idx):
        u[m[i]] = temp_u[i]
    
    # Extract Interpolated Field
    R = T1 * u.real[:, None] + T2 * u.imag[:, None]
    return R
    

In [8]:
R = construct_vector_field(v, f, tt, cf, c)
plot_mesh_field(v, f, R, cf)
print(R)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

[[ 0.23591958 -0.2272005   0.39047189]
 [-0.42277571  0.13051135 -0.14899519]
 [ 0.68709978 -0.24579441 -0.29458214]
 ...
 [ 0.44584006 -0.09987585  0.05341989]
 [ 0.34755951 -0.66810857  0.60711695]
 [ 0.4087346  -0.67078033  0.56895471]]


# Part 2 - Scalar Field Design

In [9]:
def construct_scalar_field(
    vertices: list[Any],
    faces: list[Any],
    interpolated_field: list[Any]
) -> tuple[list[Any], list[Any]]:
    # Compute Area and gradients
    gradients = igl.grad(vertices, faces)
    area = igl.doublearea(vertices, faces) / 2
    
    # Create weight matrix
    weights = sp.diags(np.concatenate((area, area, area)))
    
    # Compute u_t    
    u = np.concatenate((
        interpolated_field[:, 0],
        interpolated_field[:, 1],
        interpolated_field[:, 2]
    ))
    
    # Calculate b & K by expanding least square formula
    b = (-2 * gradients.transpose().dot(weights).dot(u))[1: ]
    K = 2 * (gradients.transpose().dot(weights).dot(gradients))[1:, 1:]
    
    # Solve for s
    s = np.concatenate((np.array([0]), sp.linalg.spsolve(K, -b)))
    
    # Compute gradient
    gradient = gradients.dot(s).reshape((3, faces.shape[0])).transpose()
    s = s.reshape(s.shape[0], 1)
    
    return s, gradient
    


In [10]:
s, g = construct_scalar_field(v, f, R)
avg = igl.avg_edge_length(v, f)/2
B = igl.barycenter(v, f)

print(s)

# Vector Field
p = mp.plot(v, f, c=s)
p.add_lines(B, B + R * avg, shading={"line_color": "black"})

# Gradient Field 
p = mp.plot(v, f, c=s)
p.add_lines(B, B + g * avg, shading={"line_color": "white"})

# Possion Error
p = mp.plot(v, f, c=s)
p.add_lines(B + R * avg, B + g * avg, shading={"line_color": "black"})


[[ 0.        ]
 [-0.76446078]
 [ 0.91827024]
 [ 0.96196406]
 [ 0.04628406]
 [-0.03356961]
 [ 0.1166635 ]
 [ 0.0375994 ]
 [ 1.06933835]
 [ 0.10824325]
 [ 0.34406749]
 [ 0.99538273]
 [ 0.60028518]
 [ 0.37920076]
 [ 0.26212548]
 [-0.38586703]
 [-0.30455482]
 [ 0.09887735]
 [ 0.14013563]
 [-0.06500472]
 [ 0.47010329]
 [ 0.31292944]
 [ 0.58217911]
 [ 0.18454032]
 [ 1.01814364]
 [ 0.97105447]
 [ 1.05592144]
 [ 0.50693882]
 [ 0.59984643]
 [ 0.53276802]
 [ 0.31799488]
 [ 0.73893937]
 [ 0.57953849]
 [ 0.19878317]
 [ 0.1723607 ]
 [ 0.00907722]
 [ 0.05661388]
 [-0.12403086]
 [ 0.14186179]
 [ 0.21680932]
 [ 0.09211278]
 [-0.03750692]
 [-0.12641178]
 [-0.58201624]
 [ 0.14224591]
 [ 0.12540661]
 [ 0.1469902 ]
 [ 0.195992  ]
 [ 0.08153612]
 [-0.15066812]
 [-0.20635038]
 [-0.50386682]
 [-0.19136987]
 [ 0.00517524]
 [ 0.05415595]
 [ 0.12011937]
 [ 0.09515583]
 [ 0.08696624]
 [ 0.11197622]
 [-0.07971778]
 [-0.007735  ]
 [-0.01119985]
 [-0.00718443]
 [ 0.10580208]
 [-0.31369651]
 [-0.45287251]
 [-0.33441

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

1

# Part 3a - Harmonic Parameterization

In [11]:
def plot_parameterization(para_type: Callable) -> None:
    v, f = igl.read_triangle_mesh("data/camel_head.off")
    uv, g_u, g_v = para_type(v, f)

    p = mp.subplot(v, f, uv=uv, s=[1, 3, 0])
    mp.subplot(v, f, uv=uv[:, 0], s=[1, 3, 1], data=p) 
    mp.subplot(v, f, uv=uv[:, 1], s=[1, 3, 2], data=p) 

    avg = igl.avg_edge_length(v, f)/2
    B = igl.barycenter(v, f)

    # Gradient of the function u
    p = mp.plot(v, f, c=uv[:, 0])
    p.add_lines(B, B + g_u * avg)

    # Gradient of the function v
    p = mp.plot(v, f, c=uv[:, 1])
    p.add_lines(B, B + g_v * avg)

    # Flattened Mesh
    mp.plot(uv, f, shading={"wireframe": True})


In [12]:
def harmonic_parameterization(
    vertices: list[Any],
    faces: list[Any]
) -> tuple[list[Any], list[Any], list[Any]]:
    # Calculate open boundary
    open_boundary = igl.boundary_loop(faces)
    
    ## Map the boundary to a circle, preserving edge proportions
    boundary_uv = igl.map_vertices_to_circle(vertices, open_boundary)
    
    ## Harmonic parametrization for the internal vertices
    uv = igl.harmonic_weights(vertices, faces, open_boundary, boundary_uv, 1)
    
    # Compute gradients
    gradients = igl.grad(vertices, faces).dot(uv)
    
    g_u = gradients[:, 0].reshape((3, faces.shape[0])).transpose()
    g_v = gradients[:, 1].reshape((3, faces.shape[0])).transpose()
    
    return uv, g_u, g_v


In [13]:
plot_parameterization(harmonic_parameterization)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(2.3782253…

# Part 3b. LSCM Parameterizations

In [14]:
def lscm_parameterization(
    vertices: list[Any], faces: list[Any]
) -> tuple[list[Any], list[Any], list[Any]]:
    
    # Fix two points on the boundary
    b = np.array([2, 1])

    open_boundary = igl.boundary_loop(faces)
    b[0] = open_boundary[0]
    b[1] = open_boundary[int(open_boundary.size / 2)]

    bc = np.array([[0.0, 0.0], [1.0, 0.0]])

    # LSCM parametrization
    _, uv = igl.lscm(vertices, faces, b, bc)
    
    # Compute Gradients
    gradients = igl.grad(vertices, faces).dot(uv)
    
    g_u = gradients[:, 0].reshape((3, faces.shape[0])).transpose()
    g_v = gradients[:, 1].reshape((3, faces.shape[0])).transpose()
    
    return uv, g_u, g_v

In [15]:
plot_parameterization(lscm_parameterization)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.031860…

# Part 4a - Editing a parameterization with vector fields

In [16]:
def edit_parameterization(
    scalar_field: list[Any], uv: list[Any]
) -> tuple[list[Any], list[Any]]:
    
    # Replace U and V with scalar_field
    uv_u = np.hstack([scalar_field, uv[:, 1].reshape(scalar_field.size, 1)])
    uv_v = np.hstack([uv[:, 0].reshape(scalar_field.size, 1), scalar_field])
    
    return uv_u, uv_v


def plot_edit_para(para_type: Callable) -> None:
    v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
    tt, _ = igl.triangle_triangle_adjacency(f)

    c = np.loadtxt("data/irr4-cyl2.constraints")
    cf = c[:, 0].astype(np.int64)
    c = c[:, 1:]
    
    # Get Intepolated field, scalar field, and gradient
    R = construct_vector_field(v, f, tt, cf, c)
    scalar_field, gradient = construct_scalar_field(v, f, R)
    
    # Get Parameterization
    uv, _, _ = para_type(v, f)
    
    # Edit Parameterization 
    uv_u, uv_v = edit_parameterization(scalar_field, uv)
    
    # Plot default mesh with gradient vectors
    avg = igl.avg_edge_length(v, f)/2
    B = igl.barycenter(v, f)
    p = mp.plot(v, f, c=s, shading={"wireframe":True})
    p.add_lines(B, B + gradient * avg)

    # Plot edited parameterization
    p = mp.subplot(v, f, uv=uv_u, s=[1,2,0])
    mp.subplot(v, f, uv=uv_v, s=[1,2,1], data=p)

In [17]:
plot_edit_para(harmonic_parameterization)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [18]:
plot_edit_para(lscm_parameterization)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Part 4b - Detecting problems with the parameterization

In [19]:
def flip_triangles(
    faces: list[Any], uv: list[Any]
) -> tuple[list[Any], list[Any]]:
    # Variables
    indices = list()
    colors = np.ones(faces.shape)
    
    for idx, face in enumerate(faces):
        v1, v2, v3 = uv[face[0]], uv[face[1]], uv[face[2]]
        
        Y = np.array([ [v1[0], v2[0], v3[0]],
                       [v1[1], v2[1], v3[1]],
                       [1.   , 1.   , 1.   ] ])
        
        # Update colors and index
        if np.linalg.det(Y) < 0:
            colors[idx, 1:] = 0
            indices.append(idx)
    
    return np.array(indices), colors

In [20]:
def plot_flipped():
    v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
    tt, _ = igl.triangle_triangle_adjacency(f)

    c = np.loadtxt("data/irr4-cyl2.constraints")
    cf = c[:, 0].astype(np.int64)
    c = c[:, 1:]

    # Get Intepolated field, scalar field, and gradient
    R = construct_vector_field(v, f, tt, cf, c)
    scalar_field, gradient = construct_scalar_field(v, f, R)
    
    # Get Parameterization
    uv, _, _ = harmonic_parameterization(v, f)
    
    # Replace V with scalar_field
    uv_v = np.hstack([uv[:, 0].reshape(scalar_field.size, 1), scalar_field])
    flipped_idx, colors = flip_triangles(f, uv_v)
    
    # Not sure if this is the right plot
    mp.plot(uv_v, f, c=colors, shading={"wireframe": True})
    
    print(flipped_idx)
    
plot_flipped()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.15…

[  0   1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  26  27  28  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  90  95  96  97  98  99 100 101
 102 103 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175
 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193
 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211
 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 26